In [14]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import VarianceThreshold

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')

Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "CodeDecision", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = train['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction","CodeDecision", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

In [15]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

# clf = ExtraTreesClassifier(n_estimators=50)
# clf = clf.fit(Xtrain, Ytrain)
# clf.feature_importances_

# model = SelectFromModel(clf, prefit=True)

# Xtrain = Xtrain[Xtrain.columns[model.get_support(indices = True)]]
# Xtest = Xtest[Xtest.columns[model.get_support(indices = True)]]

# Xtrain = model.transform(train_100k)

In [16]:
from xgboost import XGBRFClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import BorderlineSMOTE


# nbFoldValid = 5 # nombre de groupes pour la K-CV

# models2={#'SVC': SVC(),
#        'RandomForest': RandomForestClassifier(random_state=0),
#        'XGBRFClassifier' : XGBRFClassifier(random_state=0),
#        'GradientBoosting' : GradientBoostingClassifier(random_state=0),
#         'GradientBoosting' : GradientBoostingClassifier(random_state=0)
#        }

In [17]:
# Application des algorithmes 

def applyAlgo(algo, Xtrain, Ytrain, Xtest, Ytest):

    # On commence par indiquer ce que l'on va faire avec chaque algorithme.
    # on prendra soin de préciser les hyper-paramètres dont dépend l'algorithme

    if algo == "SVC":
        clf = SVC()
        clf.fit(Xtrain, Ytrain)
        rankTrain = clf.predict(Xtrain)
        rankTest = clf.predict(Xtest)

    elif algo == "RandomForest":
        clf = RandomForestClassifier(random_state=0)
        clf.fit(Xtrain, Ytrain)
        rankTest = clf.predict(Xtest)

    elif algo == "GradientBoostingClassifier":
        clf = GradientBoostingClassifier(random_state=0)
        clf.fit(Xtrain, Ytrain)
        rankTest = clf.predict(Xtest)
        
    elif algo == "AdaBoostClassifier":
        clf = AdaBoostClassifier(random_state=0)
        clf.fit(Xtrain, Ytrain)
        rankTest = clf.predict(Xtest)
        
    elif algo == "XGBRFClassifier":
        clf = XGBRFClassifier(random_state=0)
        clf.fit(Xtrain, Ytrain)
        rankTest = clf.predict(Xtest)

   # Cette deuxième partie permet d'indiquer quelle est la mesure de performance que 
   # vous souhaitez considérer pour votre étude en cours
    ctest = confusion_matrix(Ytest, rankTest)
    ftest = round(2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0]),4)

    return (ftest)

In [18]:
def Echantillon(number, Xtrain, Ytrain):
    Xtrain = Xtrain.sample(n = number)
    Ytrain = Ytrain.sample(n = number)

    return number, Xtrain, Ytrain

def Processing(method, Xtrain, Ytrain, Xtest):

    if method == "NoProcessing":
        pass
    
    if method == "SelectFromModel": 
        clf = ExtraTreesClassifier(n_estimators=50)
        clf = clf.fit(Xtrain, Ytrain)

        model = SelectFromModel(clf, prefit=True)

        Xtrain = Xtrain[Xtrain.columns[model.get_support(indices = True)]]
        Xtest = Xtest[Xtest.columns[model.get_support(indices = True)]]

    if method == "Variance": 
        selector = VarianceThreshold(threshold = 0.8)
        selector.fit_transform(Xtrain)

        Xtrain = Xtrain[Xtrain.columns[selector.get_support(indices = True)]]
        Xtest = Xtest[Xtest.columns[selector.get_support(indices = True)]]

    return method, Xtrain, Xtest

In [21]:
def SMOTE_S(sampling, Xtrain, Ytrain):

    sm = BorderlineSMOTE(sampling_strategy = sampling,random_state = 0)

    Xtrain, Ytrain = sm.fit_resample(Xtrain, Ytrain)
    
    return sampling, Xtrain, Ytrain


In [22]:
from sklearn.pipeline import Pipeline
# pipe_Sampling = Pipeline([('Smote', BorderlineSMOTE()), ('Modele',model)])
# pipe_Sampling = Pipeline([('Smote', BorderlineSMOTE()), ('Modele',model)])

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import average_precision_score, confusion_matrix
# from functions import loadCsv, oneHotEncodeColumns, data_recovery
from sklearn.preprocessing import Normalizer

# Normalisation
# normalizer = Normalizer()
# normalizer.fit(Xtrain)
# Xtrain = normalizer.transform(Xtrain)
# Xtest = normalizer.transform(Xtest)

In [23]:
models = ['RandomForest', 'XGBRFClassifier' , 'GradientBoostingClassifier', 'AdaBoostClassifier']# 'SVC']
number = [100000, 380000 , 999999, 3899362]
method = ['NoProcessing', 'SelectFromModel' , 'Variance']
sampling = [0.15, 0.3 , 0.5]

import time
Fmesure = []
execution = []
algo = []
n_data = []
variable_selector = []
smote = []

columns = ["Classifieur", "Echantillon", "Selection Variable", "Performance", "Sampling", "Temps_execution"] # rajouter pipeline ici si on veut comparer les pré processing

for s in sampling :
    for x in number :
        for l in method : 
            for model in models : 

                s, Xtrain, Ytrain = SMOTE_S(s, Xtrain, Ytrain)
                smote.append(s)

                x, X_sample_train, Y_sample_train = Echantillon(x, Xtrain, Ytrain)
                n_data.append(x)
                
                l, X_sample_train, X_sample_test = Processing(l, X_sample_train, Y_sample_train, Xtest)
                variable_selector.append(l)

                # Stocker l'algorithme qui tourne
                algo.append(model)
                start = time.time()
                # Fin normalisation
                apTest = applyAlgo(model, X_sample_train, Y_sample_train, X_sample_test, Ytest)
                Fmesure.append(apTest) 
                end = time.time()
                elapsed = end - start 
                execution.append(elapsed)


score_df = pd.DataFrame(list(zip(algo, n_data, variable_selector, Fmesure, smote, execution)), columns= columns) 
score_df.to_csv("resultats_smote.csv")